In [ ]:
%%capture
%pip install pandas matplotlib plotly nbformat ipywidgets scipy kaleido

In [ ]:
import time
import datetime
import requests
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import nbformat
import numpy as np
from scipy.signal import find_peaks

nbformat.current_nbformat

def find_peak_value(array):
    """
    Find the maximum peak value in a 1D array. E.g.:
    peak_value, peak_index = find_peak_value(data)
    
    Args:
        array (array-like): 1D numeric array
        
    Returns:
        tuple: (peak_value, peak_index)
    """
    # Convert input to numpy array if it isn't already
    array = np.array(array)
    
    # Find all peaks
    peak_indices, _ = find_peaks(array)
    
    # If no peaks found, return the maximum value and its index
    if len(peak_indices) == 0:
        max_index = np.argmax(array)
        return array[max_index], max_index
    
    # Find the highest peak
    peak_values = array[peak_indices]
    max_peak_idx = np.argmax(peak_values)
    
    return peak_values[max_peak_idx], peak_indices[max_peak_idx]


def find_nearest_value(array, value):
  """Finds the nearest value in the array to the given value."""
  array = np.asarray(array)
  idx = (np.abs(array - value)).argmin()
  return array[idx]

## Plot Spectra

In [ ]:
avgspecdf = pd.read_csv("data/led_spectra_202412071247.csv.gz")
peakdf = pd.read_csv("data/led_peaks_202412071247.csv")

In [30]:
# Constants for plots below
colors = ['#636EFA', '#EF553B']
FONTCOLOR = 'rgba(0.4,0.4,0.4,1.0)'
GRIDCOLOR = 'rgba(0.85,0.85,0.9,0.5)'
FONTSIZE = 16
# Cursor chat query :
# create a list of 15 html hex colors that roughly match the perceved color of the following 
# wavelengths of light (in nanometers): [270,405,448,470,505,530,568,591,617,627,655,740,850,940]. 
# Please use python syntax.
cmap = [
    '#4B0082',  # ~270nm (UV approximated as deep indigo)
    '#8B00FF',  # ~405nm (violet)
    '#0000FF',  # ~448nm (blue)
    '#0080FF',  # ~470nm (light blue)
    '#00FF90',  # ~505nm (blue-green)
    '#00FF00',  # ~530nm (green)
    '#9FEE00',  # ~568nm (yellow-green)
    '#DDDD00',  # ~591nm (yellow)
    '#FF8000',  # ~617nm (orange)
    '#FF4000',  # ~627nm (orange-red)
    '#FF0000',  # ~655nm (red)
    '#CC0000',  # ~740nm (deep red)
    '#990000',  # ~850nm (IR approximated as very deep red)
    '#660000',  # ~940nm (IR approximated as darkest visible red)
]

In [ ]:
fig = px.line(avgspecdf, x="nm", y="spec", color="led", color_discrete_sequence=cmap,
             labels={"nm": 'Wavelength (nm)', "spec": "Spectrometer Voltage (mv)", "led": "LED ID"})
fig.update_layout(width=850, height=480, font=dict(size=FONTSIZE, color=FONTCOLOR), 
                  paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
                  xaxis_gridcolor=GRIDCOLOR, yaxis_gridcolor=GRIDCOLOR,
                  legend=dict(font_size=FONTSIZE - 2)) #yanchor="top", y=0.99, xanchor="right", x=0.95

fig.update_yaxes(showgrid=True, showline=True, zerolinecolor=GRIDCOLOR)
fig.update_xaxes(showgrid=True, showline=True, zerolinecolor=GRIDCOLOR)
fig.write_image("multispec_spectra.png", scale=2)
fig.show()

In [ ]:
print(peakdf.to_string(index=False))

In [ ]:
specdf = pd.read_csv("led_spectra_raw_202412071247.csv.gz")
print(specdf.shape)
specdf.head()

In [ ]:
led = "led591" #"led405" "led530" "led568" "led591"

fig = px.line(specdf.loc[specdf.led == led], x="nm", y="spec", color="integ",
             labels={"nm": 'Wavelength (nm)', "spec": "Spectrometer Voltage (mv)"})
fig.update_layout(width=850, height=480, font=dict(size=FONTSIZE, color=FONTCOLOR), 
                  paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
                  xaxis_gridcolor=GRIDCOLOR, yaxis_gridcolor=GRIDCOLOR,
                  legend=dict(font_size=FONTSIZE - 2)) #yanchor="top", y=0.99, xanchor="right", x=0.95

fig.update_yaxes(showgrid=True, showline=True, zerolinecolor=GRIDCOLOR)
fig.update_xaxes(showgrid=True, showline=True, zerolinecolor=GRIDCOLOR)
#fig.write_image("multispec_spectra_reproducible.png", scale=2)
fig.show()

## Measure Spectra

In [ ]:
SPECURL = "http://192.168.0.248"
r = requests.get(f"{SPECURL}/wavelength")
nm = r.json()["nm"]
xrange = [np.floor(min(nm))-1, np.ceil(max(nm))+1]
leds = [270,405,448,470,505,530,568,591,617,627,655,740,850,940]
specs = {nm:[] for nm in leds}

### Set integration time
Run the following to set the integration time for the current LED to be measured. The spectral peak is used to guess which LED has been measured. However, for some edge-cases the peak is misleading. E.g., the 560nm phosphor LED has a peak around 530nm (the spec of 560nm is the center-of-mass of this LED's broad spectrum). The peak for LEDs with peaks outside the spectrometer's specified range of 340nm - 850nm (the actual range is dependent on the calibration parameters for each device, but should be about 310 - 883

In [ ]:
intusec = 1000
lednm_override = None
MIN_VAL, MAX_VAL = 500, 2800

data = requests.get(f"{SPECURL}/spectrum/{intusec}").json()
pk, pkidx = find_peak_value(data["spec"])
pknm = nm[pkidx]
led_nm = find_nearest_value(leds, pknm) if lednm_override is None else lednm_override
fig = px.line(x=nm, y=data["spec"], labels={"nm": 'Wavelength (nm)', "spec": "Voltage (mv)"})
fig.update_layout(width=850, height=480, font=dict(size=FONTSIZE, color=FONTCOLOR), 
                  paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
                  xaxis_gridcolor=GRIDCOLOR, yaxis_gridcolor=GRIDCOLOR,
                  legend=dict(font_size=FONTSIZE - 2)) #yanchor="top", y=0.99, xanchor="right", x=0.95
fig.update_yaxes(showgrid=True, showline=True, zerolinecolor=GRIDCOLOR)
fig.update_xaxes(showgrid=True, range=xrange, showline=True, zerolinecolor=GRIDCOLOR)
readtimes = data["readtimes"]
print(f"actual integration time={readtimes[2] - readtimes[0]} usec (readtimes={readtimes})")
print(f"{led_nm=}, {pknm=}, {pk=}")
fig.show()

In [ ]:
n = 30
for cnt in range(n):
    data = requests.get(f"{SPECURL}/spectrum/{intusec}").json()
    pk, pkidx = find_peak_value(data["spec"])
    y = data["spec"]
    if pk < MIN_VAL or pk > MAX_VAL:
        data = requests.get(f"{SPECURL}/spectrum/{intusec}").json()
        pk, pkidx = find_peak_value(data["spec"])
        if pk < MIN_VAL or pk > MAX_VAL:
            print(f"peak voltage {pk} our of range; stopping.")
            break
    pknm = nm[pkidx]
    #led_nm = find_nearest_value(leds, pknm)
    led_nm = find_nearest_value(leds, pknm) if lednm_override is None else lednm_override
    #if led_nm != target:
    #    print(f"Peak does not match target {pknm=}, {target=}, {led_nm=}")
    #    break
    specs[led_nm].append(data)
    print(f"{led_nm=}, pknm={nm[pkidx]}, {pk=}, readtimes={data['readtimes']}")
    time.sleep(0.25)


## Run the following to save data

In [ ]:
print(" ".join([f"{k}:{len(v)}" for k,v in specs.items()]))
ts = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# Save the raw json payloads
with open(f"multispec_{ts}.json", "wt") as fp:
    json.dump(specs, fp)
    
# Save data in dataframes
led_specs = []
led_specs_avg = []
led_peaks = []
for peaknm, d in specs.items():
    avg_spec = np.zeros(len(nm))
    ledid = f"led{peaknm}"
    for mnum, d in enumerate(d):
        integ = d["integration"]
        aspec = d["spec"]
        avg_spec += np.array(aspec)
        led_specs += [{"led":ledid, "integ":integ, "nm":n, "spec":s}
                   for n,s in zip(nm, aspec)]
    avg_spec /= mnum + 1
    pk, pkidx = find_peak_value(avg_spec)
    led_peaks.append({"led":ledid, "peak_spec":peaknm, "peak_measured":nm[pkidx]})
    led_specs_avg += [{"led":ledid, "nm":n, "spec":s} for n,s in zip(nm, avg_spec)]
           
ledpeakdf = pd.DataFrame(led_peaks)
specdf = pd.DataFrame(led_specs)
avgspecdf = pd.DataFrame(led_specs_avg)
ledpeakdf.to_csv(f"led_peaks_{ts}.csv", index=False)
specdf.to_csv(f"led_spectra_raw_{ts}.csv", index=False)
avgspecdf.to_csv(f"led_spectra_{ts}.csv", index=False)
print(ledpeakdf)
print(avgspecdf.shape)
avgspecdf.head()